# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f221cb45eb0>
├── 'a' --> tensor([[ 1.2228e-03,  4.7870e-01, -6.9829e-01],
│                   [-5.2456e-01, -1.2414e+00,  1.0046e+00]])
└── 'x' --> <FastTreeValue 0x7f221cb45e80>
    └── 'c' --> tensor([[-0.6458,  0.3453,  0.5015, -1.4360],
                        [ 1.1610,  1.8588, -1.5224,  0.7069],
                        [ 0.1793,  0.7413,  0.7598,  1.2943]])

In [4]:
t.a

tensor([[ 1.2228e-03,  4.7870e-01, -6.9829e-01],
        [-5.2456e-01, -1.2414e+00,  1.0046e+00]])

In [5]:
%timeit t.a

68 ns ± 0.041 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f221cb45eb0>
├── 'a' --> tensor([[-0.0043,  1.5404, -2.0569],
│                   [-0.1538,  0.3693,  1.2714]])
└── 'x' --> <FastTreeValue 0x7f221cb45e80>
    └── 'c' --> tensor([[-0.6458,  0.3453,  0.5015, -1.4360],
                        [ 1.1610,  1.8588, -1.5224,  0.7069],
                        [ 0.1793,  0.7413,  0.7598,  1.2943]])

In [7]:
%timeit t.a = new_value

65.6 ns ± 0.0269 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.2228e-03,  4.7870e-01, -6.9829e-01],
               [-5.2456e-01, -1.2414e+00,  1.0046e+00]]),
    x: Batch(
           c: tensor([[-0.6458,  0.3453,  0.5015, -1.4360],
                      [ 1.1610,  1.8588, -1.5224,  0.7069],
                      [ 0.1793,  0.7413,  0.7598,  1.2943]]),
       ),
)

In [10]:
b.a

tensor([[ 1.2228e-03,  4.7870e-01, -6.9829e-01],
        [-5.2456e-01, -1.2414e+00,  1.0046e+00]])

In [11]:
%timeit b.a

62.8 ns ± 0.0119 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.8646, -2.1097,  0.1872],
               [-1.1104, -0.2046, -1.8591]]),
    x: Batch(
           c: tensor([[-0.6458,  0.3453,  0.5015, -1.4360],
                      [ 1.1610,  1.8588, -1.5224,  0.7069],
                      [ 0.1793,  0.7413,  0.7598,  1.2943]]),
       ),
)

In [13]:
%timeit b.a = new_value

482 ns ± 0.177 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

848 ns ± 14.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 19.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 296 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 1.58 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f221cb458b0>
├── 'a' --> tensor([[[ 1.2228e-03,  4.7870e-01, -6.9829e-01],
│                    [-5.2456e-01, -1.2414e+00,  1.0046e+00]],
│           
│                   [[ 1.2228e-03,  4.7870e-01, -6.9829e-01],
│                    [-5.2456e-01, -1.2414e+00,  1.0046e+00]],
│           
│                   [[ 1.2228e-03,  4.7870e-01, -6.9829e-01],
│                    [-5.2456e-01, -1.2414e+00,  1.0046e+00]],
│           
│                   [[ 1.2228e-03,  4.7870e-01, -6.9829e-01],
│                    [-5.2456e-01, -1.2414e+00,  1.0046e+00]],
│           
│                   [[ 1.2228e-03,  4.7870e-01, -6.9829e-01],
│                    [-5.2456e-01, -1.2414e+00,  1.0046e+00]],
│           
│                   [[ 1.2228e-03,  4.7870e-01, -6.9829e-01],
│                    [-5.2456e-01, -1.2414e+00,  1.0046e+00]],
│           
│                   [[ 1.2228e-03,  4.7870e-01, -6.9829e-01],
│                    [-5.2456e-01, -1.2414e+00,  1.0046e+00]],
│           
│  

In [21]:
%timeit t_stack(trees)

32.5 µs ± 74.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f221db55460>
├── 'a' --> tensor([[ 1.2228e-03,  4.7870e-01, -6.9829e-01],
│                   [-5.2456e-01, -1.2414e+00,  1.0046e+00],
│                   [ 1.2228e-03,  4.7870e-01, -6.9829e-01],
│                   [-5.2456e-01, -1.2414e+00,  1.0046e+00],
│                   [ 1.2228e-03,  4.7870e-01, -6.9829e-01],
│                   [-5.2456e-01, -1.2414e+00,  1.0046e+00],
│                   [ 1.2228e-03,  4.7870e-01, -6.9829e-01],
│                   [-5.2456e-01, -1.2414e+00,  1.0046e+00],
│                   [ 1.2228e-03,  4.7870e-01, -6.9829e-01],
│                   [-5.2456e-01, -1.2414e+00,  1.0046e+00],
│                   [ 1.2228e-03,  4.7870e-01, -6.9829e-01],
│                   [-5.2456e-01, -1.2414e+00,  1.0046e+00],
│                   [ 1.2228e-03,  4.7870e-01, -6.9829e-01],
│                   [-5.2456e-01, -1.2414e+00,  1.0046e+00],
│                   [ 1.2228e-03,  4.7870e-01, -6.9829e-01],
│                   [-5.2456e-01, -1.2414e+00,  1.0046

In [23]:
%timeit t_cat(trees)

30.1 µs ± 56.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.9 µs ± 76.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.6458,  0.3453,  0.5015, -1.4360],
                       [ 1.1610,  1.8588, -1.5224,  0.7069],
                       [ 0.1793,  0.7413,  0.7598,  1.2943]],
              
                      [[-0.6458,  0.3453,  0.5015, -1.4360],
                       [ 1.1610,  1.8588, -1.5224,  0.7069],
                       [ 0.1793,  0.7413,  0.7598,  1.2943]],
              
                      [[-0.6458,  0.3453,  0.5015, -1.4360],
                       [ 1.1610,  1.8588, -1.5224,  0.7069],
                       [ 0.1793,  0.7413,  0.7598,  1.2943]],
              
                      [[-0.6458,  0.3453,  0.5015, -1.4360],
                       [ 1.1610,  1.8588, -1.5224,  0.7069],
                       [ 0.1793,  0.7413,  0.7598,  1.2943]],
              
                      [[-0.6458,  0.3453,  0.5015, -1.4360],
                       [ 1.1610,  1.8588, -1.5224,  0.7069],
                       [ 0.1793,  0.7413,  0.7598,  1.2943]],

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 135 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.6458,  0.3453,  0.5015, -1.4360],
                      [ 1.1610,  1.8588, -1.5224,  0.7069],
                      [ 0.1793,  0.7413,  0.7598,  1.2943],
                      [-0.6458,  0.3453,  0.5015, -1.4360],
                      [ 1.1610,  1.8588, -1.5224,  0.7069],
                      [ 0.1793,  0.7413,  0.7598,  1.2943],
                      [-0.6458,  0.3453,  0.5015, -1.4360],
                      [ 1.1610,  1.8588, -1.5224,  0.7069],
                      [ 0.1793,  0.7413,  0.7598,  1.2943],
                      [-0.6458,  0.3453,  0.5015, -1.4360],
                      [ 1.1610,  1.8588, -1.5224,  0.7069],
                      [ 0.1793,  0.7413,  0.7598,  1.2943],
                      [-0.6458,  0.3453,  0.5015, -1.4360],
                      [ 1.1610,  1.8588, -1.5224,  0.7069],
                      [ 0.1793,  0.7413,  0.7598,  1.2943],
                      [-0.6458,  0.3453,  0.5015, -1.4360],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 1.15 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

315 µs ± 449 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
